This example shows how to solve the [FAST quick start example](https://web.stanford.edu/~lcambier/fast/demo.php) using this package.
We start by setting the different constants:

In [1]:
const num_stages = 2
const numScen = 2
const C = 1
const P = 2
const d = [2, 3];

We now model the master problem using [StructJuMP](https://github.com/StructJuMP/StructJuMP.jl).

In [2]:
using StructJuMP
m1 = StructuredModel(num_scenarios=numScen)
@variable(m1, x >= 0)
@objective(m1, Min, C * x);

For each of the two scenarios we need to create a [StructJuMP](https://github.com/StructJuMP/StructJuMP.jl) model specifying that `m1` is the parent and that the scenario has probability `1/2`.

In [3]:
for ξ in 1:numScen
    m2 = StructuredModel(parent=m1, prob=1/2, id=ξ)
    @variable(m2, s >= 0)
    @constraints m2 begin
        s <= d[ξ]
        s <= x
    end
    @objective(m2, Max, P * s)
end

We first need to pick an LP solver, see [here](http://www.juliaopt.org/) for a list of the available choices.

In [4]:
using GLPKMathProgInterface
solver = GLPKMathProgInterface.GLPKSolverLP();

This structured model need to be transformed into an appropriate structure to run SDDP on it.
This is achieved by `stochasticprogram`:

In [5]:
using CutPruners
const pruner = AvgCutPruningAlgo(-1)
using StructDualDynProg
const SOI = StructDualDynProg.StochOptInterface
sp = SOI.stochasticprogram(m1, num_stages, solver, pruner);

We can now run the sddp algorithm on it using `SDDP`. We are using 2 forward paths per iteration and we stop either after 10 iterations or once the pereira criterion is satisfied with $\alpha = 0.1$.

In [6]:
algo = StructDualDynProg.SDDP.Algorithm(K = 2)
sol = SOI.optimize!(sp, algo, SOI.Pereira(0.1) | SOI.IterLimit(10))

StructDualDynProg.StochOptInterface.SolutionSummary(:Optimal, -2.0, [2.0], Dict{Any,Any}(Pair{Any,Any}(:stats, Lower Bound: -2.0
Upper Bound: -2.0
),Pair{Any,Any}(:nfcuts, 0),Pair{Any,Any}(:nocuts, 5),Pair{Any,Any}(:niter, 4)))

We can verify that the algorithm have found the right value by inspecting the solution:

In [7]:
sol.objval

-2.0